In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_absolute_error
import sklearn
import math
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV,Lasso
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
frame=pd.read_excel("COVID-19.xlsx")
frame.drop(["geoId","day","month","year","countryterritoryCode"],axis=1,inplace=True)
frame.columns=[["Date","Confirmed","Deaths","Country","Population"]]
frame.drop("Population",axis=1,inplace=True)
frame.columns = frame.columns.get_level_values(0)
frame["Date"]=pd.to_datetime(frame["Date"],format="%d/%m/%Y")
frame.head()


,Date,Confirmed,Deaths,Country
0,2020-04-10,61,1,Afghanistan
1,2020-04-09,56,3,Afghanistan
2,2020-04-08,30,4,Afghanistan
3,2020-04-07,38,0,Afghanistan
4,2020-04-06,29,2,Afghanistan


In [3]:
frame.dtypes


Date         datetime64[ns]
Confirmed             int64
Deaths                int64
Country              object
dtype: object

In [4]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9922 entries, 0 to 9921
Data columns (total 4 columns):
Date         9922 non-null datetime64[ns]
Confirmed    9922 non-null int64
Deaths       9922 non-null int64
Country      9922 non-null object
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 310.1+ KB


In [5]:
frame=frame.sort_values(by=["Country","Date"])
frame.head()

,Date,Confirmed,Deaths,Country
91,2019-12-31,0,0,Afghanistan
90,2020-01-01,0,0,Afghanistan
89,2020-01-02,0,0,Afghanistan
88,2020-01-03,0,0,Afghanistan
87,2020-01-04,0,0,Afghanistan


In [6]:
frame["Total_Confirmed"]=frame.groupby("Country")["Confirmed"].cumsum()
frame["Total_Deaths"]=frame.groupby("Country")["Deaths"].cumsum()



In [7]:
df_Bangladesh=frame[frame["Country"]=="Bangladesh"]
df_world=frame[frame["Country"]!="Bangladesh"]

In [8]:
df_Bangladesh.tail()

,Date,Confirmed,Deaths,Country,Total_Confirmed,Total_Deaths
882,2020-04-06,35,1,Bangladesh,123,9
881,2020-04-07,41,3,Bangladesh,164,12
880,2020-04-08,54,5,Bangladesh,218,17
879,2020-04-09,112,3,Bangladesh,330,20
878,2020-04-10,94,1,Bangladesh,424,21


In [9]:
df_world=df_world[df_world["Total_Confirmed"]>0]

In [10]:
df_world['Days'] = (df_world['Date'] - df_world.groupby('Country')['Date'].transform('first'))
df_Bangladesh['Days'] = (df_Bangladesh['Date'] - df_Bangladesh.groupby('Country')['Date'].transform('first'))

In [11]:
df_Bangladesh

,Date,Confirmed,Deaths,Country,Total_Confirmed,Total_Deaths,Days
904,2020-03-09,3,0,Bangladesh,3,0,0 days
903,2020-03-15,0,0,Bangladesh,3,0,6 days
902,2020-03-17,2,0,Bangladesh,5,0,8 days
901,2020-03-18,3,0,Bangladesh,8,0,9 days
900,2020-03-19,2,0,Bangladesh,10,0,10 days
899,2020-03-20,0,0,Bangladesh,10,0,11 days
898,2020-03-21,7,0,Bangladesh,17,0,12 days
897,2020-03-22,7,2,Bangladesh,24,2,13 days
896,2020-03-23,3,1,Bangladesh,27,3,14 days
895,2020-03-24,6,0,Bangladesh,33,3,15 days


In [12]:
a=df_world["Country"].unique()

In [13]:
df_world["Days"]=df_world["Days"].astype(str)
df_Bangladesh["Days"]=df_Bangladesh["Days"].astype(str)
df_world["Days"] = df_world["Days"].str.split(' ').str[0]
df_Bangladesh["Days"] = df_Bangladesh["Days"].str.split(' ').str[0]
df_world["Days"]=df_world["Days"].astype(int)
df_Bangladesh["Days"]=df_Bangladesh["Days"].astype(int)
df_world["Days"]=df_world["Days"]+1
df_Bangladesh["Days"]=df_Bangladesh["Days"]+1

In [14]:
df_USA=df_world[df_world["Country"]=="United_States_of_America"]
df_USA=df_USA[["Days","Total_Confirmed"]]
df_UK=df_world[df_world["Country"]=="United_Kingdom"]
df_UK=df_UK[["Days","Total_Confirmed"]]
df_Italy=df_world[df_world["Country"]=="Italy"]
df_Italy=df_Italy[["Days","Total_Confirmed"]]
df_Spain=df_world[df_world["Country"]=="Spain"]
df_Spain=df_Spain[["Days","Total_Confirmed"]]
df_France=df_world[df_world["Country"]=="France"]
df_France=df_France[["Days","Total_Confirmed"]]

df_India=df_world[df_world["Country"]=="India"]
df_India=df_India[["Days","Total_Confirmed"]]
df_Pakistan=df_world[df_world["Country"]=="Pakistan"]
df_Pakistan=df_Pakistan[["Days","Total_Confirmed"]]
df_Bangladesh=df_Bangladesh[["Days","Total_Confirmed"]]

In [15]:
df_USA = df_USA.sample(frac=1,random_state=42).reset_index(drop=True)
df_UK = df_UK.sample(frac=1,random_state=42).reset_index(drop=True)
df_Italy = df_Italy.sample(frac=1,random_state=42).reset_index(drop=True)
df_France = df_France.sample(frac=1,random_state=42).reset_index(drop=True)
df_Spain = df_Spain.sample(frac=1,random_state=42).reset_index(drop=True)
df_India = df_India.sample(frac=1,random_state=42).reset_index(drop=True)
df_Pakistan = df_Pakistan.sample(frac=1,random_state=42).reset_index(drop=True)
df_Bangladesh = df_Bangladesh.sample(frac=1,random_state=42).reset_index(drop=True)



In [16]:
df_USA["EXTRA"]=1
df_UK["EXTRA"]=1
df_Italy["EXTRA"]=1
df_France["EXTRA"]=1
df_Spain["EXTRA"]=1
df_India["EXTRA"]=1
df_Bangladesh["EXTRA"]=1
df_Pakistan["EXTRA"]=1

In [17]:
df_USA_X=df_USA[["Days","EXTRA"]]
df_USA_Y=df_USA["Total_Confirmed"]

df_UK_X=df_UK[["Days","EXTRA"]]
df_UK_Y=df_UK["Total_Confirmed"]

df_Italy_X=df_Italy[["Days","EXTRA"]]
df_Italy=df_Italy["Total_Confirmed"]

df_France_X=df_France[["Days","EXTRA"]]
df_France_Y=df_France["Total_Confirmed"]

df_Spain_X=df_Spain[["Days","EXTRA"]]
df_Spain_Y=df_Spain["Total_Confirmed"]

df_India_X=df_India[["Days","EXTRA"]]
df_India_Y=df_India["Total_Confirmed"]

df_Pakistan_X=df_Pakistan[["Days","EXTRA"]]
df_Pakistan_Y=df_Pakistan["Total_Confirmed"]

df_Bangladesh_X=df_Bangladesh[["Days","EXTRA"]]
df_Bangladesh_Y=df_Bangladesh["Total_Confirmed"]

In [18]:
data = {'days':[34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61]} 
df_Bangladesh_Test=pd.DataFrame(data)
df_Bangladesh_Test["EXTRA"]=1
df_Bangladesh_Test.head()

,days,EXTRA
0,34,1
1,35,1
2,36,1
3,37,1
4,38,1


In [19]:
##from sklearn.kernel_approximation import RBFSampler
#rbf_feature = RBFSampler(gamma=1, random_state=1)
#X_features_USA = rbf_feature.fit_transform(df_USA_X)
#X_features_Bangladesh = rbf_feature.fit_transform(df_Bangladesh_X)
#df_Bangladesh_Test= rbf_feature.fit_transform(df_Bangladesh_Test)

In [24]:
def predict_Linear(X,y,df_USA_X,df_USA_Y,df_Bangladesh_Test,poly=None):
    x_train, x_test, y_train, y_test = train_test_split(X, y ,test_size =0.2,random_state=42, shuffle=False)
    model=LinearRegression()
    if (poly==True):
        poly = PolynomialFeatures(degree=6)
        x_train = poly.fit_transform(x_train)
        X = poly.fit_transform(X)
        x_test = poly.fit_transform(x_test)
        df_USA_X = poly.fit_transform(df_USA_X)
        df_Bangladesh_Test = poly.fit_transform(df_Bangladesh_Test)
        
    model.fit(x_train,y_train)
    predict_linear=model.predict(x_test)
    mse_linear = sklearn.metrics.mean_squared_error(y_test, predict_linear)
    rmse_linear = math.sqrt(mse_linear)
    from sklearn.metrics import r2_score,mean_absolute_error
    print("r2_score_linear",r2_score(y_test, predict_linear))
    print("mean_absolute_error_linear",mean_absolute_error(y_test, predict_linear))
    model.fit(X,y)
    predict_future_linear=model.predict(df_Bangladesh_Test)    
    
    return mse_linear,rmse_linear,predict_linear,y_test,predict_future_linear

In [25]:
mse_linear,rmse_linear,predict_linear,y_test,predict_future_linear= predict_Linear(df_Bangladesh_X,df_Bangladesh_Y,df_USA_X,df_USA_Y,df_Bangladesh_Test,poly=True)


r2_score_linear 0.9941459090266078
mean_absolute_error_linear 1.1578785170108048


In [26]:
rmse_linear

1.4418791085749807

In [27]:
mse_linear

2.079015363744981

In [28]:
predict_linear=predict_linear.astype(int)
predict_linear

array([72, 22, 37, 49, 60, 16])

In [29]:
y_test

21    70
22    24
23    39
24    48
25    61
26    17
Name: Total_Confirmed, dtype: int64

In [30]:
predict_future_linear=predict_future_linear.astype(int)
predict_future_linear

array([  582,   779,  1029,  1342,  1728,  2201,  2774,  3463,  4283,
        5255,  6397,  7731,  9283, 11077, 13142, 15508, 18208, 21277,
       24753, 28677, 33092, 38044, 43583, 49761, 56635, 64264, 72710,
       82042])

In [31]:
def predict_Lasso(X,y,df_USA_X,df_USA_Y,df_Bangladesh_Test,poly=None):
    x_train, x_test, y_train, y_test = train_test_split(X, y ,test_size =0.2,random_state=42, shuffle=False)
    model=Lasso(alpha=0.001,random_state=42)
    if (poly==True):
        poly = PolynomialFeatures(degree=8)
        x_train = poly.fit_transform(x_train)
        X = poly.fit_transform(X)
        x_test = poly.fit_transform(x_test)
        df_USA_X = poly.fit_transform(df_USA_X)
        df_Bangladesh_Test = poly.fit_transform(df_Bangladesh_Test)
        
    rmse_Lasso_cv = np.sqrt(-cross_val_score(Lasso(alpha=0.58), x_train, y_train, scoring="neg_mean_squared_error", cv = 5))
    model.fit(x_train,y_train)
    predict_Lasso=model.predict(x_test)
    mse_Lasso = sklearn.metrics.mean_squared_error(y_test, predict_Lasso)
    rmse_Lasso = math.sqrt(mse_Lasso)
    from sklearn.metrics import r2_score,mean_absolute_error
    print("r2_score_Lasso",r2_score(y_test,predict_Lasso))
    print("mean_absolute_error_Lasso",mean_absolute_error(y_test, predict_Lasso))
    model.fit(X,y)
    predict_future_Lasso=model.predict(df_Bangladesh_Test)    
    
    #r2_score=model.score(y_test, predict)
    return mse_Lasso,rmse_Lasso,predict_Lasso,y_test,predict_future_Lasso,rmse_Lasso_cv

In [32]:
mse_Lasso,rmse_Lasso,predict_Lasso,y_test,predict_future_Lasso,rmse_Lasso_cv= predict_Lasso(df_Bangladesh_X,df_Bangladesh_Y,df_USA_X,df_USA_Y,df_Bangladesh_Test,poly=True)


r2_score_Lasso 0.9846224006179275
mean_absolute_error_Lasso 1.8247129672748983


In [33]:
rmse_Lasso_cv

array([11.41424146,  4.57839922,  8.12263706,  1.55648191, 23.68955505])

In [34]:
rmse_Lasso_cv.mean()

9.872262942035924

In [35]:
rmse_Lasso

2.3369175334888674

In [36]:
mse_Lasso

5.461183558327691

In [37]:
predict_Lasso=predict_Lasso.astype(int)
predict_Lasso

array([73, 25, 38, 47, 61, 20])

In [38]:
y_test

21    70
22    24
23    39
24    48
25    61
26    17
Name: Total_Confirmed, dtype: int64

In [39]:
predict_future_Lasso=predict_future_Lasso.astype(int)
predict_future_Lasso

array([   583,    784,   1042,   1372,   1786,   2301,   2938,   3718,
         4666,   5810,   7185,   8825,  10771,  13071,  15776,  18943,
        22635,  26925,  31889,  37617,  44202,  51750,  60377,  70210,
        81388,  94061, 108396, 124572])